<a href="https://colab.research.google.com/github/Zilleplus/MachineLearning/blob/feature%2Fadd-HML-chap15-notebooks/HML/chapter15/1DConvolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
print("using tensorflow version: " + tf.__version__)

def generate_time_series(batch_size, n_steps):
    freq1, freq2, offset1, offset2, = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offset1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offset2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)


n_steps = 50
n_samples = 10000
series = generate_time_series(batch_size=n_samples, n_steps=n_steps + 1)
x_train, y_train = series[:7000, :n_steps], series[:7000, -1]
x_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
x_test, y_test = series[9000:, :n_steps], series[9000:, -1]

using tensorflow version: 2.7.0


In [14]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model = keras.models.Sequential([ 
  keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid"),
  keras.layers.GRU(20, return_sequences=True),
  keras.layers.GRU(20, return_sequences=True),
  keras.layers.TimeDistributed(keras.layers.Dense(1))
  ])
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=[last_time_step_mse])
# The book predicts 10 values, I simplified the example and only predict 1 value.
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 11s 28ms/step - loss: 0.1442 - last_time_step_mse: 0.1422 - val_loss: 0.1383 - val_last_time_step_mse: 0.1353
Epoch 2/20
219/219 [==============================] - 5s 24ms/step - loss: 0.1405 - last_time_step_mse: 0.1388 - val_loss: 0.1358 - val_last_time_step_mse: 0.1348
Epoch 3/20
219/219 [==============================] - 7s 31ms/step - loss: 0.1400 - last_time_step_mse: 0.1382 - val_loss: 0.1370 - val_last_time_step_mse: 0.1344
Epoch 4/20
219/219 [==============================] - 6s 27ms/step - loss: 0.1400 - last_time_step_mse: 0.1378 - val_loss: 0.1356 - val_last_time_step_mse: 0.1324
Epoch 5/20
219/219 [==============================] - 5s 24ms/step - loss: 0.1392 - last_time_step_mse: 0.1370 - val_loss: 0.1359 - val_last_time_step_mse: 0.1373
Epoch 6/20
219/219 [==============================] - 5s 25ms/step - loss: 0.1396 - last_time_step_mse: 0.1368 - val_loss: 0.1353 - val_last_time_step_mse: 0.1334
Epoch 7/20
219/219 [=

In [9]:
y_train.shape

(7000, 1)

In [15]:
x_train.shape

(7000, 50, 1)